In [1]:
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

(train_images, train_labels),(test_images, test_labels)= keras.datasets.mnist.load_data() 

image_size = 27
train_images = np.expand_dims(train_images, axis = -1)
train_images = tf.image.resize(train_images,[image_size,image_size])
train_images = np.squeeze(train_images)
train_images= train_images/255.0

test_images = np.expand_dims(test_images, axis = -1)
test_images = tf.image.resize(test_images,[image_size,image_size])
test_images = np.squeeze(test_images)
test_images= test_images/255.0

for k in range(int(train_images.shape[0])) :
      for i in range(image_size):
            for j in range(image_size):
                if train_images[k, i,j] <0.3:
                    train_images[k, i, j] = 0
                elif train_images[k, i,j] <0.7:
                    train_images[k, i,j] = 0.5
                else :
                    train_images[k, i,j] = 1

for k in range(int(test_images.shape[0])) :
  for i in range(image_size):
    for j in range(image_size):
      if test_images[k, i,j] <0.3:
        test_images[k, i, j] = 0
      elif test_images[k, i,j] <0.7:
        test_images[k, i,j] = 0.5
      else :
        test_images[k, i,j] = 1

train = [[],[],[],[],[],[],[],[],[],[]]
for k in range(train_images.shape[0]):
  if train_labels[k] ==0:
    train[0].append(train_images[k])
  elif train_labels[k] ==1:
    train[1].append(train_images[k])
  elif train_labels[k] ==2:
    train[2].append(train_images[k])
  elif train_labels[k] ==3:
    train[3].append(train_images[k])
  elif train_labels[k] ==4:
    train[4].append(train_images[k])
  elif train_labels[k] ==5:
    train[5].append(train_images[k])
  elif train_labels[k] ==6:
    train[6].append(train_images[k])
  elif train_labels[k] ==7:
    train[7].append(train_images[k])
  elif train_labels[k] ==8:
    train[8].append(train_images[k])
  elif train_labels[k] ==9:
    train[9].append(train_images[k])
  else:
    train_images[k] = train_images[k]

2023-04-11 15:08:29.793800: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 15:08:29.932094: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-11 15:08:30.546377: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-04-11 15:08:30.546456: W tensorflow/compiler/xla/stream_exe

In [2]:
#6개 edge 선언
filters = [[],[],[],[],[],[]]
filters[0] = [[1,2,1],[0,0,0],[-1,-2,-1]]#0
filters[1] = [[0,1,3],[0,0,0],[-3,-1,0]] #30
filters[2] = [[0,0,3],[-1,0,1],[-3,0,0]] #60
filters[3] = [[-1,0,1],[-2,0,2],[-1,0,1]]#90
filters[4] = [[-3,0,0],[-1,0,1],[0,0,3]] #120
filters[5] = [[-3,-1,0],[0,0,0],[0,1,3]] #150

In [3]:
def flattening(image,filter):
  edge_sum = [[0]*(9) for _ in range(9)]
  flat_result_sum=[[0] for _ in range(81)]
  for i in range(0, image.shape[0]-2, 3):
    for j in range(0, image.shape[1]-2, 3):
      image_patch = image[i:i+3, j:j+3]
      edge_sum[int(i/3)][int(j/3)]=np.sum(np.multiply(image_patch,filter))
  
  flat_result_sum=[element for row in edge_sum for element in row]
  return flat_result_sum

In [4]:
flat_result_sum=[[],[],[],[],[],[]]
for i in range(6):
  flat_result_sum[i] = flattening(train_images[0],filters[i])

In [5]:
def edge_pattern(flattened_sum):
  
  temp = []
  max_index=[[0] for _ in range(81)]
  edge_pattern=[[0] for _ in range(80)]
  
    
  for i in range(81):
    temp.clear()
    for j in range(6):
      temp.append(flattened_sum[j][i])

    if(max(temp)==0):
      max_index[i] = 20
    elif(max(temp)==temp[0]):
      max_index[i] = 0
    elif(max(temp)==temp[1]):
      max_index[i] = 1
    elif(max(temp)==temp[2]):
      max_index[i] = 2
    elif(max(temp)==temp[3]):
      max_index[i] = 3
    elif(max(temp)==temp[4]):
      max_index[i] = 4
    elif(max(temp)==temp[5]):
      max_index[i] = 5
    else:
      max_index[i] = 20
      
  for i in range(80):
    if(max_index[i]==20):
      if(max_index[i+1]==20):
        edge_pattern[i]=0
      else:
        #edge_pattern[i]=5
        edge_pattern[i]=max_index[i+1]
    elif(max_index[i+1]==20):
      #edge_pattern[i]=5
      edge_pattern[i]=max_index[i]
    else:
      edge_pattern[i] = max_index[i]-max_index[i+1]
        
  return edge_pattern

In [6]:
def dtw_distance(signal1, signal2):
    # 시계열 데이터의 길이를 저장합니다.
    n, m = len(signal1), len(signal2)
    
    # DTW 행렬을 생성합니다.
    dtw = np.zeros((n+1, m+1))
    for i in range(1, n+1):
        dtw[i][0] = np.inf
    for i in range(1, m+1):
        dtw[0][i] = np.inf
    dtw[0][0] = 0
    
    # DTW 행렬을 계산합니다.
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(signal1[i-1] - signal2[j-1])
            dtw[i][j] = cost + min(dtw[i-1][j], dtw[i][j-1], dtw[i-1][j-1])
    
    # 최종적인 DTW 거리를 반환합니다.
    return dtw[-1][-1]

In [7]:
import time
from tqdm import tqdm
import pandas as pd

In [8]:
# test_images 를 모두 1차원 벡터화한다. 
#train_images_flat = [[] for _ in range(10)]
train_images_flat = []
temp_1 = [[],[],[],[],[],[]]
temp_2 = [[] for _ in range(80)]

#for k in tqdm(range(10)):
for i in tqdm(range(10000)):
  for j in range(6):
    temp_1[j] = flattening(test_images[i],filters[j])
  temp_2 = edge_pattern(temp_1)
  train_images_flat.append(temp_2)

# 1. train 이미지를 0부터 9까지 101개씩 평면화하기.
# 2. 0번 이미지 와 나머지 1~100번 이미지 까지 DTW 구해서 평균값을 구해서 표로 만들기 

100%|██████████| 10000/10000 [00:45<00:00, 220.32it/s]


In [9]:
print(train_images_flat[4])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0, 0, 0, 5, 5, 0, 5, 5, 0, 0, 0, 4, 4, 0, 4, 4, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 1, -4, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
from random import *
class chip_random:
    def __init__(self, size, threshold1 = 0.9, threshold2 = 0.95):
        '''
        size = memory input size 
        threshold1 = don't care  역치 //  0.9
        threshold2 = 1 역치 // 0.95
        '''
        self.size = size
        self.threshold1 = threshold1
        self.threshold2 = threshold2
        self.cam = []
        self.counter = []

        for i in range(2**size-1):
            a = randint(1,2**size-1)
            while a in self.cam:
                a = randint(1,2**size-1)
            self.cam.append(a)
            self.counter.append(0)

    def reset(self):
        self.cam = []
        self.counter = []

        for i in range(2**self.size-1):
            a = randint(1,2**self.size-1)
            while a in self.cam:
                a = randint(1,2**self.size-1)
            self.cam.append(a)
            self.counter.append(0)

    def pattern_to_number(self, pattern):
        num = 0
        result = []
        for i in range(len(pattern)):
            if pattern[i]==1:
                num = num + 2**(len(pattern)-1-i)
        result.append(num)
        for i in range(len(pattern)):
            if pattern[i]==0.5:
                for k in range(len(result)):
                    result.append(result[k]+2**(len(pattern)-1-i))
    
        return result

    def change_threshold(self, threshold1, threshold2):
        self.threshold1 = threshold1
        self.threshold2 = threshold2

    def learn(self, pattern):
        num = self.pattern_to_number(pattern)
        for i in range(len(self.cam)):
            if num.count(self.cam[i])!=0:
                self.counter[i] = self.counter[i] +1

        for i in range(len(num)):
            for j in range(len(self.cam)-1):
                if self.counter[j] > self.counter[j+1]:
                    temp =self.counter[j]
                    self.counter[j] = self.counter[j+1]
                    self.counter[j+1] = temp
                    temp =self.cam[j]
                    self.cam[j] = self.cam[j+1]
                    self.cam[j+1] = temp
    
    def address_up(self, pattern):
        num = self.pattern_to_number(pattern)
        for i in range(len(self.cam)-2,0,-1):
            if self.cam[i] in num:
                temp = self.cam[i]
                self.cam[i] = self.cam[i+1]
                self.cam[i+1] = temp

    def address_down(self, pattern):
        num = self.pattern_to_number(pattern)
        for i in range(len(self.cam)-2):
            if self.cam[i] in num:
                temp = self.cam[i]
                self.cam[i] = self.cam[i-1]
                self.cam[i-1] = temp


    def get_address(self, pattern):
        num = self.pattern_to_number(pattern)
        result = []
        for i in range(len(num)):
            if num[i] ==0:
                result.append(0)
            else :
                result.append(self.cam.index(num[i]))
        return result
  
    def get_binary(self, pattern):
        result = self.get_address(pattern)
        address = max(result)
        percent = address/len(self.cam)

        if percent < self.threshold1 :
            return 0.0
        elif percent <self.threshold2:
            return 0.5
        else:
            return 1.0

In [11]:
class layer_random:
    def __init__(self, window_size, layer_size, stride = 0, threshold1 = 0.9, threshold2 = 0.95):
        self.L = [[0]*layer_size for _ in range(layer_size)]
        self.threshold1 = threshold1
        self.threshold2 = threshold2
        for i in range(layer_size):
            for j in range(layer_size):
                self.L[i][j] = chip_random(window_size*window_size, self.threshold1, self.threshold2)
        
        if stride ==0:
            self.stride = window_size
        else :
            self.stride = stride
        self.window_size = window_size
        self.layer_size = layer_size

    def reset(self):
        '''
        layer가 초기화됨 내부 chip의 배열도 초기화
        '''
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                self.L[i][j].reset()

    def change_threshold(self, threshold1, threshold2):
        self.threshold1 = threshold1
        self.threshold2 = threshold2
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                self.L[i][j].change_threshold(threshold1, threshold2)

    def learn(self, image):
        '''
        이미지 하나를 layer가 입력받게 되는 함수
        '''
        # 학습 가능한 이미지 사이즈인지 확인하는 코드 추가 작성
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                pattern= []
                for a in range(self.window_size):
                    for b in range(self.window_size):
                        pattern.append(image[self.stride*i+a][self.stride*j+b])
                self.L[i][j].learn(pattern)
    
    def address_up(self, image):
        '''
        이미지 안에 있는 패턴을 주소 1 up
        '''
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                pattern = []
                for a in range(self.window_size):
                    for b in range(self.window_size):
                        pattern.append(image[self.stride*i+a][self.stride*j+b])
                self.L[i][j].address_up(pattern)
            
    def address_down(self, image):
        '''
        이미지 안에 있는 패턴을 주소 1 up
        '''
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                pattern = []
                for a in range(self.window_size):
                    for b in range(self.window_size):
                        pattern.append(image[self.stride*i+a][self.stride*j+b])
                self.L[i][j].address_down(pattern)
    
    def out(self, image):
        '''
        이미지 넣고 출력되는 layer out 보여줌
        '''
        # 입력 가능한 이미지 사이즈인지 확인하는 코드 추가 작성
        out_image = [[0] * self.layer_size for _ in range(self.layer_size)]
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                pattern=[]
                for a in range(self.window_size):
                    for b in range(self.window_size):
                        pattern.append(image[self.stride*i +a][self.stride*j +b])
                out_image[i][j] = self.L[i][j].get_binary(pattern)
        return out_image

    def out_sum(self, image):
        '''
        이미지 넣고 출력되는 이미지의 전체 합 출력
        '''
        out_image = self.out(image)
        result = 0
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                result = result + out_image[i][j]
        return result
    
    def weighted_outsum (self, image):
        out_image = self.out(image)
        result = 0
        weight1 = 0
        weight2 = 0
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                if (i>self.layer_size/3)*(i<2*self.layer_size/3) : weight1 = 2
                else : weight1 = 1

                if(j>self.layer_size/3)*(j<2*self.layer_size/3) : weight2 = 2
                else : weight2 = 1

                result = result + weight1 * weight2 * out_image[i][j]
        return result

    def out_address(self,image):
        '''
        마지막 레이어의 주소를 반환
        '''
        out_image = [[0] * self.layer_size for _ in range(self.layer_size)]
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                pattern=[]
                for a in range(self.window_size):
                    for b in range(self.window_size):
                        pattern.append(image[self.stride*i +a][self.stride*j +b])
                out_image[i][j] = max(self.L[i][j].get_address(pattern))
        return out_image

    def get_avg_address(self,image):
        
        out_image = self.out_address(image)
        sum = 0
        for i in range (self.layer_size):
            for j in range(self.layer_size):
                sum = sum + out_image[i][j]

        return sum/(self.layer_size*self.layer_size)
    
    def num_to_pattern(self, num):
        temp = num
        pattern = [0 for _ in range(self.window_size*self.window_size)]
        for i in range(len(pattern)):
            if(num%2) : pattern[i] = 1
            else : pattern[i] = 0
            num = num//2
        pattern.reverse()
        return pattern

    def feature(self, depth):
        out_image = [[0] * self.layer_size *self.window_size for _ in range(self.layer_size*self.window_size)]
        pattern = []
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                for k in range(depth):
                    pattern = self.num_to_pattern(self.L[i][j].cam[-1-k])
                    for a in range(self.window_size):
                        for b in range(self.window_size):
                            out_image[self.window_size*i + a ][self.window_size*j + b ] += pattern[self.window_size*a + b ] * 2**(depth -k-1)
        
        return out_image

    def out_size(self):
        '''
        N*N 출력 이미지에서 N이 나옴 
        '''
        return self.layer_size

# LG

In [12]:
print(len(train_images_flat[5]))

80


In [13]:
Layer_test = [0 for _ in tqdm(range(1000))]
for i in tqdm(range(1000)):
  Layer_test[i] = layer_random(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)

counter = [0 for _ in tqdm(range(1000))]
flat_last = [0 for _ in tqdm(range(1000))]
#100개의 layer 

# 그중 100개미리 학습, 이때 각 러닝그룹당 flat 의 저장 공간 역시 존재해야 함
for i in tqdm(range(1000)):
  Layer_test[i].learn(test_images[i])
  #counter[i] += 1
  flat_last[i] = train_images_flat[i]  

100%|██████████| 1000/1000 [00:31<00:00, 31.55it/s]


In [15]:
dtw_temp = [0 for _ in tqdm(range(100))]
min_dtw_index = 0

for i in tqdm(range(200)):
  dtw_temp = [0 for _ in range(100)]
  min_dtw_index = 0
  for k in range(10):
    for j in range(100): 
      dtw_temp[j] = dtw_distance(flat_last[j],train_images_flat[i+100*k])
    min_dtw_index = dtw_temp.index(min(dtw_temp))
    Layer_test[min_dtw_index].learn(train_images[i+100*k])
    counter[min_dtw_index] += 1 
    flat_last[i] = train_images_flat[i+100*k]

 52%|█████▏    | 104/200 [16:39<15:22,  9.61s/it]


KeyboardInterrupt: 

In [18]:
dtw_temp = [0 for _ in tqdm(range(100))]
min_dtw_index = 0
lgh = [[] for _ in tqdm(range(100))]

for i in tqdm(range(100)):
  dtw_temp = [0 for _ in range(100)]
  min_dtw_index = 0
  for k in range(10):
    for j in range(100): 
      dtw_temp[j] = dtw_distance(flat_last[j],train_images_flat[i+100*k])
      #lgh[j].append(train_labels[i+100*k])
    min_dtw_index = dtw_temp.index(min(dtw_temp))
    Layer_test[min_dtw_index].learn(train_images[i+100*k])
    lgh[min_dtw_index].append(train_labels[i+100*k])
    counter[min_dtw_index] += 1 
    flat_last[i] = train_images_flat[i+100*k]

100%|██████████| 100/100 [15:51<00:00,  9.52s/it]


In [ ]:
for i in range()

In [19]:
for i in range(100):
  print(lgh[i])

[0, 1, 0, 7, 3, 5, 5, 7]
[2, 1]
[0, 0, 0, 8, 9, 0, 7, 9, 3, 2, 6, 7, 4, 8, 1, 1, 6]
[5, 7, 9]
[7, 8]
[8, 1, 5, 2, 5, 3, 5, 9, 5, 9, 1, 4, 8, 1, 5, 3, 9, 2, 5, 7, 7, 2]
[3, 7, 2, 4, 7, 6, 3, 0, 2, 6, 9, 6, 3, 1]
[9, 2, 1, 1, 1, 1, 7, 8, 2, 0, 1, 3, 9, 6, 0, 2, 0, 4, 1, 5, 8, 4, 0, 4, 9, 1, 4, 3, 8, 6, 6, 6, 3, 9, 7, 3, 1, 6, 5, 7, 0, 9, 3, 8, 3, 7, 3, 8]
[9, 1, 4, 2, 3, 3, 1]
[6, 7, 6]
[5, 8, 4, 1, 7, 2, 9, 6, 3, 1, 6, 3, 6, 7, 4, 3, 8, 4, 0, 5]
[9, 0, 7, 2, 5, 0, 9, 9, 8, 9, 0, 3, 1, 8, 8]
[2, 1, 8, 1, 7, 6, 0, 0, 4]
[4, 4, 1, 7]
[3]
[0, 1]
[4, 5, 8, 5, 5, 6, 4, 9]
[8, 0, 5, 0, 4, 4, 6, 8, 7, 5, 6, 6]
[9, 3, 5, 1, 2, 9, 7, 0, 9, 3, 5, 1, 4, 7, 9, 4, 9, 4, 4, 9, 3, 2, 3, 0, 2, 4, 5, 0, 2]
[5, 5]
[1, 9, 2, 8, 7, 7, 1, 3, 9, 9, 7, 9, 0, 1, 2, 9]
[5, 8, 0]
[2, 8, 6, 1, 5, 8, 8, 7, 8, 8, 6, 1, 6, 7, 6, 2, 1, 7, 7, 9, 5, 7, 3, 1, 6, 3]
[3, 5, 9, 4, 8, 5, 5, 2, 6]
[4, 1, 5, 6, 4, 2, 5, 6, 8, 1, 1, 8]
[6]
[9, 8]
[0, 2, 9, 4, 3, 5, 2, 5, 7, 0, 9, 2, 9, 7, 3, 0, 4, 0, 8, 6, 6, 6, 8, 1, 8, 3, 4, 

In [20]:
dtw_temp = [0 for _ in tqdm(range(100))]
min_dtw_index = 0
lgh_1 = [[] for _ in tqdm(range(100))]

for i in tqdm(range(200)):
  dtw_temp = [0 for _ in range(100)]
  min_dtw_index = 0
  for k in range(10):
    for j in range(100): 
      dtw_temp[j] = dtw_distance(flat_last[j],train_images_flat[i+100*k])
      #lgh[j].append(train_labels[i+100*k])
    min_dtw_index = dtw_temp.index(min(dtw_temp))
    Layer_test[min_dtw_index].learn(train_images[i+100*k])
    lgh_1[min_dtw_index].append(train_labels[i+100*k])
    counter[min_dtw_index] += 1 
    flat_last[i] = train_images_flat[i+100*k]

100%|██████████| 200/200 [31:54<00:00,  9.57s/it]


In [21]:
for i in range(100):
  print(lgh_1[i])

[0, 1, 0, 7, 3, 5, 5, 7, 9, 0, 7, 3, 5, 5, 7]
[2, 1, 6, 2, 1]
[0, 0, 0, 8, 9, 0, 7, 9, 3, 2, 6, 7, 4, 8, 1, 1, 6, 9, 1, 0, 0, 8, 0, 2, 2, 6, 1, 7, 8, 1, 1, 6]
[5, 7, 9, 0, 7]
[7, 8, 7, 7, 8]
[8, 1, 5, 2, 5, 3, 5, 9, 5, 9, 1, 4, 8, 1, 5, 3, 9, 2, 5, 7, 7, 2, 8, 1, 1, 5, 2, 5, 3, 5, 9, 5, 9, 2, 4, 8, 1, 9, 5, 9, 2, 7, 0, 2]
[3, 7, 2, 4, 7, 6, 3, 0, 2, 6, 9, 6, 3, 1, 4, 7, 2, 4, 7, 6, 2, 3, 0, 8, 2, 6, 9, 6]
[9, 2, 1, 1, 1, 1, 7, 8, 2, 0, 1, 3, 9, 6, 0, 2, 0, 4, 1, 5, 8, 4, 0, 4, 9, 1, 4, 3, 8, 6, 6, 6, 3, 9, 7, 3, 1, 6, 5, 7, 0, 9, 3, 8, 3, 7, 3, 8, 9, 1, 2, 1, 1, 7, 8, 0, 7, 7, 1, 9, 6, 0, 2, 4, 1, 5, 8, 4, 0, 4, 9, 1, 4, 3, 8, 6, 6, 9, 7, 7, 3, 1, 5, 7, 6, 0, 9, 3, 8, 3, 7, 4, 8]
[9, 1, 4, 2, 3, 3, 1, 9, 7, 1, 4, 2, 3, 3, 1]
[6, 7, 6, 3, 6, 7]
[5, 8, 4, 1, 7, 2, 9, 6, 3, 1, 6, 3, 6, 7, 4, 3, 8, 4, 0, 5, 5, 8, 6, 7, 2, 9, 6, 3, 1, 6, 3, 0, 6, 1, 7, 4, 3, 3, 0, 5]
[9, 0, 7, 2, 5, 0, 9, 9, 8, 9, 0, 3, 1, 8, 8, 9, 0, 7, 2, 5, 7, 0, 9, 9, 8, 9, 0, 3, 1, 8, 8]
[2, 1, 8, 1, 7, 6, 0, 0, 4, 5, 

In [22]:
for i in range(100):
  if((i+1)%20 == 0):
    print(counter[i])
  else:
    print(counter[i], end = '  ')

61  15  100  18  14  150  96  310  50  21  131  100  54  33  14  24  54  84  190  16
106  23  164  66  74  11  15  232  105  98  94  41  18  37  41  180  58  5  35  15
125  94  47  204  78  186  222  45  11  55  69  4  132  22  77  40  16  14  151  56
58  17  99  123  21  17  167  66  11  21  98  51  82  15  176  7  51  157  57  62
44  63  67  29  61  46  32  28  34  85  30  16  27  38  32  8  34  26  53  11


In [ ]:
for i in range(100):
  if((i+1)%20 == 0):
    print(counter[i])
  else:
    print(counter[i], end = '  ')
  

In [23]:
print(sum(counter))

6721


In [ ]:
for i in range(100):
  if(counter[i]>100): 
    print(i, counter[i])

In [ ]:
dtw_comparison = [[0 for _ in range(100)] for _ in range(100)]

for i in tqdm(range(100)):
  for j in range(100):
    dtw_comparison[i][j] = dtw_distance(flat_last[i],flat_last[j])
    

In [ ]:
print(dtw_comparison[0])

In [ ]:
print(dtw_comparison[2])